# 13.3章 追跡確定能力の予測

追跡システムは、最終段で確定されたトラックを表示するに至るまでに、追跡信頼度における幾つかの段階が定義できる。この時、確定されたトラックのみが運用者に表示される。ここで、誤確定トラックの生成確率を最小化することが重要である。

In [27]:
%matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.precision = 2

In [2]:
data = np.array([
    [0.8, 1.5e-5,  4.9,  5.3,  7.1, None, None,  4.4,  5.0,  5.3,  8.0],
    [0.7, 4.1e-5,  7.5,  8.0, 11.8, None, None,  8.7, 12.5,  8.8, 14.6],
    [0.7, 2.8e-4, 14.0, 14.1, 21.1, None, None, 14.0, 20.5, 21.6, 43.0],
    [0.6, 2.2e-4, 22.4, 19.2, 31.5, None, None, 17.5, 25.0, 35.0, 73.0],
])

df_paper = pd.DataFrame(data)
df_label = []
df_label.extend(['PD', 'PFA', 'SPRT AveTc'])
df_label.extend(['SMC MHT AveTc', 'SMC MHT T90'])
df_label.extend(['FMC GNN AveTc', 'FMC GNN T90'])
df_label.extend(['FMC MHT AveTc', 'FMC MHT T90'])
df_label.extend(['FMC IPDA AveTc', 'FMC IPDA T90'])
df_paper.columns = df_label

In [23]:
%%time
import IRSTexample

def calc_TcT90(PD, PFA, n_scan):
    eval = IRSTexample.generate_irst_example_p878(PD, PFA)
    res = eval.estimate_track_statistics(n_scan=n_scan, n_run=100)
    row = pd.Series( [PD, PFA, None, None, None, res["Tc"][0], res["T90"][0], None, None, None, None], index=df_paper.columns)
    return row

row_1 = calc_TcT90(0.8, 1.5e-5, 10)
row_2 = calc_TcT90(0.7, 4.1e-5, 10)
row_3 = calc_TcT90(0.7, 2.8e-4, 20)
row_4 = calc_TcT90(0.6, 2.2e-4, 20)

df_sim = pd.DataFrame(columns=df_paper.columns)
df_sim = df_sim.append(row_1, ignore_index=True)
df_sim = df_sim.append(row_2, ignore_index=True)
df_sim = df_sim.append(row_3, ignore_index=True)
df_sim = df_sim.append(row_4, ignore_index=True)

Wall time: 21min


確定時間の期待値(Tc)と90パーセンタイル値(T90)について、文献と本プログラムで比較検証した。  （上段：文献値、下段：計算値）  
＊＊＊  
＊＊＊  
＊＊＊  

In [28]:
df_paper

,PD,PFA,SPRT AveTc,SMC MHT AveTc,SMC MHT T90,FMC GNN AveTc,FMC GNN T90,FMC MHT AveTc,FMC MHT T90,FMC IPDA AveTc,FMC IPDA T90
0,0.8,1.5e-05,4.9,5.3,7.1,None,None,4.4,5,5.3,8
1,0.7,4.1e-05,7.5,8,12,None,None,8.7,12,8.8,15
2,0.7,0.00028,14,14,21,None,None,14,20,22,43
3,0.6,0.00022,22,19,32,None,None,18,25,35,73


In [29]:
df_sim

,PD,PFA,SPRT AveTc,SMC MHT AveTc,SMC MHT T90,FMC GNN AveTc,FMC GNN T90,FMC MHT AveTc,FMC MHT T90,FMC IPDA AveTc,FMC IPDA T90
0,0.8,1.50e-05,NaN,NaN,NaN,3.51,6.00,NaN,NaN,NaN,NaN
1,0.7,4.10e-05,NaN,NaN,NaN,4.74,8.50,NaN,NaN,NaN,NaN
2,0.7,2.80e-04,NaN,NaN,NaN,7.73,13.33,NaN,NaN,NaN,NaN
3,0.6,2.20e-04,NaN,NaN,NaN,8.46,16.88,NaN,NaN,NaN,NaN
